# Web of Science Starter and Expanded APIs in Python

by Michael T. Moen and Avery Fernandez

**Web of Science Starter API Documentation**: https://api.clarivate.com/swagger-ui/?url=https%3A%2F%2Fdeveloper.clarivate.com%2Fapis%2Fwos-starter%2Fswagger%3FforUser%3D984a370b61be00d769c8d8c6ec49abba804d5a2e

**Web of Science Expanded API Documentation**: https://api.clarivate.com/swagger-ui/?url=https%3A%2F%2Fdeveloper.clarivate.com%2Fapis%2Fwos%2Fswagger

**Web of Science Examples**: https://images.webofknowledge.com/WOKRS529AR7/help/WOS/hp_advanced_examples.html

**Web of Science API Release Notes:** https://clarivate.com/academia-government/release-notes/wos-apis/

**Web of Science Terms of Use**: https://clarivate.com/wp-content/uploads/dlm_uploads/2019/08/End-User-Terms.pdf

**Clarivate Product / Service Terms**: https://clarivate.com/wp-content/uploads/dlm_uploads/2023/12/Product-Terms-v3.7-Web-of-Science-APIs.pdf

Please check with your institution on Web of Science API access.

Users from the University of Alabama (UA) have access to the [institutional plan for the WOS Starter API](https://developer.clarivate.com/apis/wos-starter#plans) and the [basic plan for the WOS Expanded API](https://developer.clarivate.com/apis/wos) and can sign up [here](https://developer.clarivate.com/). (see the [Developer Portal](https://developer.clarivate.com/apis/wos#plans) for more information).
- UA users are encouraged to access the WOS Starter API using the institutional plan, which has higher limits (5 requests/second and 5,000 requests/day) than the free plan (1 request/second and 50 requests/day).
- UA users can access the WOS Expanded API using the institution's basic plan, which limits the access of the whole institution (2 requests/second and 50,000 full records/year). Please contact UA Libraries if you anticipate a need greater than 1000 full record API requests.

We thank Clarivate Support for their helpful suggestions with this API tutorial.

*These recipe examples were tested on February 12, 2025.*

## Setup

### API Keys



Import your API keys:

In [1]:
from api_keys import wos_starter_key, wos_expanded_key

This tutorial will use the headers for authentication:

In [2]:
WOS_STARTER_HEADER = {"X-ApiKey": wos_starter_key}
WOS_EXPANDED_HEADER = {"X-ApiKey": wos_expanded_key}

### Import Libraries

This tutorial uses the following libraries:

In [3]:
import requests
from pprint import pprint
from time import sleep
import pandas as pd

## 1. Basic Search Queries with WOS Starter

To begin, let's construct a basic query to retrieve data for articles about machine learning.

The URL requires the following four parameters:

1. `db` specifies what database to use. In this case, we are using the `WOS` (Web of Science) database.
2. `q` specifies the query we are using. The following parameters are used to specify topics and document types:
    - `TS` specifies the topic to search for. In this case, we are looking for articles with "Machine Learning" in the topic.
    - `DT` spcifies the document type. In this case we are looking for articles.
3. `limit` specifies the number of records to return. Here, we use the maximum value of 50.
4. `page` specifies what page to return, allowing us to page beyond the first page of results if needed.

You can practice generating queries here: https://www.webofscience.com/wos/woscc/advanced-search

You can also read about more advanced queries here: 
* https://webofscience.help.clarivate.com/en-us/Content/search-advanced-egs.html
* https://webofscience.help.clarivate.com/en-us/Content/search-rules.htm
* https://webofscience.help.clarivate.com/en-us/Content/search-operators.html
* https://webofscience.help.clarivate.com/en-us/Content/home.htm

In [4]:
parameters = '&'.join([
    'db=WOS',
    'q=((TS="Machine Learning") AND DT=Article)',
    'limit=50',
    'page=1'
])

url = f'https://api.clarivate.com/apis/wos-starter/v1/documents?{parameters}'
response = requests.get(url, headers=WOS_STARTER_HEADER)

# Status code 200 means the call was successful
response.status_code

200

Now, let's extract the data:

In [5]:
data = response.json()

# Display first record from returned data
data['hits'][0]

{'uid': 'WOS:A1959WX53200001',
 'title': 'SOME STUDIES IN MACHINE LEARNING USING THE GAME OF CHECKERS',
 'types': ['Article'],
 'sourceTypes': ['Article'],
 'source': {'sourceTitle': 'IBM JOURNAL OF RESEARCH AND DEVELOPMENT',
  'publishYear': 1959,
  'volume': '3',
  'issue': '3',
  'pages': {'range': '211-&', 'begin': '211', 'end': '&', 'count': 0}},
 'names': {'authors': [{'displayName': 'SAMUEL, AL',
    'wosStandard': 'SAMUEL, AL',
    'researcherId': 'DQE-0158-2022'}]},
 'links': {'record': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosliteua&SrcAuth=WosAPI&KeyUT=WOS:A1959WX53200001&DestLinkType=FullRecord&DestApp=WOS_CPL',
  'references': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosliteua&SrcAuth=WosAPI&KeyUT=WOS:A1959WX53200001&DestLinkType=CitedReferences&DestApp=WOS',
  'related': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosliteua&SrcAuth=WosAPI&KeyUT=WOS:A1959WX53200001&DestLinkType=RelatedRecords&DestApp=WOS_CPL'},
 '

We are given a lot of data, but we can index out specific data:

In [6]:
# Grab authors of the sixth record
for author in data["hits"][5]["names"]["authors"]:
    print(author['displayName'])

REED, J
TOOMBS, R
BARRICELLI, NA


In [7]:
# Grab DOI of first record
data["hits"][0]["identifiers"]["doi"]

'10.1147/rd.33.0210'

We can also loop through the results to get data from different articles:

In [8]:
# Print the source title of the first five articles
for article in data["hits"][:5]:
    pprint(article["source"]["sourceTitle"])

'IBM JOURNAL OF RESEARCH AND DEVELOPMENT'
'INFORMATION AND CONTROL'
'IEEE TRANSACTIONS ON INFORMATION THEORY'
'INFORMATION AND CONTROL'
'INFORMATION AND CONTROL'


### Searching by DOI

We can use the `DO` tag in our query in order to search for an article by its DOI. In the example below, we retrieve the first article returned by the query above using its DOI.

In [9]:
# Extract the DOI from the first result
doi = data["hits"][0]["identifiers"]["doi"]

parameters = '&'.join([
    'db=WOS',
    f'q=(DO={doi})',
    'limit=1',
    'page=1'
])

url = f'https://api.clarivate.com/apis/wos-starter/v1/documents?{parameters}'
response = requests.get(url, headers=WOS_STARTER_HEADER)
doi_data = response.json()

# Note that this data is the same as what's returned above
doi_data["hits"]

[{'uid': 'WOS:A1959WX53200001',
  'title': 'SOME STUDIES IN MACHINE LEARNING USING THE GAME OF CHECKERS',
  'types': ['Article'],
  'sourceTypes': ['Article'],
  'source': {'sourceTitle': 'IBM JOURNAL OF RESEARCH AND DEVELOPMENT',
   'publishYear': 1959,
   'volume': '3',
   'issue': '3',
   'pages': {'range': '211-&', 'begin': '211', 'end': '&', 'count': 0}},
  'names': {'authors': [{'displayName': 'SAMUEL, AL',
     'wosStandard': 'SAMUEL, AL',
     'researcherId': 'DQE-0158-2022'}]},
  'links': {'record': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosliteua&SrcAuth=WosAPI&KeyUT=WOS:A1959WX53200001&DestLinkType=FullRecord&DestApp=WOS_CPL',
   'references': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosliteua&SrcAuth=WosAPI&KeyUT=WOS:A1959WX53200001&DestLinkType=CitedReferences&DestApp=WOS',
   'related': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosliteua&SrcAuth=WosAPI&KeyUT=WOS:A1959WX53200001&DestLinkType=RelatedRecords&DestAp

### Retrieving More Than 50 Results

If we want to access results beyond the first 50 records, we must modify the `page` parameter on successive calls. The example below loops through the first 300 results.

In [ ]:
parameters = '&'.join([
    'db=WOS',
    'q=((TS="Machine Learning") AND DT=Article)',
    'limit=50',
    f'page=1'
])

url = f'https://api.clarivate.com/apis/wos-starter/v1/documents?{parameters}'
response = requests.get(url, headers=WOS_STARTER_HEADER)
data = response.json()

# Print metadata
data["metadata"]

{'total': 310228, 'page': 1, 'limit': 50}

From the metadata above, we can see that this query yields 310,228 results. In order to obtain results beyond the first 50, we must modify the `page` parameter. The example below shows how to retrieve the first 300 results using 6 API calls:

In [14]:
titles = []

for i in range(1, 7):

    parameters = '&'.join([
        'db=WOS',
        'q=((TS="Machine Learning") AND DT=Article)',
        'limit=50',
        f'page={i}'
    ])

    url = f'https://api.clarivate.com/apis/wos-starter/v1/documents?{parameters}'
    response = requests.get(url, headers=WOS_STARTER_HEADER)
    data = response.json()
    for article in data["hits"]:
        titles.append(article["title"])
    sleep(1)

# Print length of the results to confirm it worked
len(titles)

300

In [15]:
# Print first 3 titles
titles[:3]

['SOME STUDIES IN MACHINE LEARNING USING THE GAME OF CHECKERS',
 'PROGRAMS FOR MACHINE LEARNING .1.',
 'PATTERN-RECOGNITION AND MACHINE LEARNING']

### Another Example with More Complex Options

In this example, we will add more specifications like publication date and document type.

The URL requires the following four parameters:

1. `db` specifies what database to use. In this case, we are using the `WOS` (Web of Science) database.
2. `q` specifies the query we are using. The following parameters are used in this case:
    - `TS` specifies the topic to search for. In this case, we use `machine learn*` to look for topics that contain the word "machine" or words that start with "learn".
    - `PY` specifies the publication year. In this case, we use `2009-2011`.
    - `DT` spcifies the document type. In this case, we are looking for articles.
3. `limit` specifies the number of records to return. Here, we use the maximum value of 50.
4. `page` specifies what page to return, allowing us to page beyond the first page of results if needed.

In [ ]:
parameters = '&'.join([
    'db=WOS',
    'q=((TS=machine learn*) AND PY=(2009-2011) AND DT=Article)',
    'limit=50',
    'page=1'
])

url = f'https://api.clarivate.com/apis/wos-starter/v1/documents?{parameters}'
response = requests.get(url, headers=WOS_STARTER_HEADER)

# Status code 200 means the call was successful
response.status_code

200

In [17]:
data = response.json()["hits"]
data[0]

{'uid': 'WOS:000261040700014',
 'title': 'Learning from dependent observations',
 'types': ['Article'],
 'sourceTypes': ['Article'],
 'source': {'sourceTitle': 'JOURNAL OF MULTIVARIATE ANALYSIS',
  'publishYear': 2009,
  'publishMonth': 'JAN',
  'volume': '100',
  'issue': '1',
  'pages': {'range': '175-194', 'begin': '175', 'end': '194', 'count': 20}},
 'names': {'authors': [{'displayName': 'Steinwart, Ingo',
    'wosStandard': 'Steinwart, I',
    'researcherId': 'KBD-2936-2024'},
   {'displayName': 'Hush, Don',
    'wosStandard': 'Hush, D',
    'researcherId': 'FBE-3544-2022'},
   {'displayName': 'Scovel, Clint',
    'wosStandard': 'Scovel, C',
    'researcherId': 'DQO-0915-2022'}]},
 'links': {'record': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosliteua&SrcAuth=WosAPI&KeyUT=WOS:000261040700014&DestLinkType=FullRecord&DestApp=WOS_CPL',
  'references': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosliteua&SrcAuth=WosAPI&KeyUT=WOS:000261040700014&De

## 2. Batch Processing with WOS Starter

This example shows a method to search for as many 50 DOIs in a single query. We suppose that we have a file `dois.txt` that contains 1000 DOIs, each listed on its own line. The DOIs in this file were obtained via the OpenAlex API (see our [OpenAlex recipes](https://ua-libraries-research-data-services.github.io/UALIB_ScholarlyAPI_Cookbook/src/overview/openalex.html) for guidance on how to do this). Note that not all DOIs in this list are indexed in Web of Science.

In [18]:
dois = []
with open("dois.txt", 'r') as f:
    for line in f:
        dois.append(line.strip())

len(dois)

1000

Since the maximum number of results we can retrieve in a single page is 50, we can retrieve the data for each DOI with 20 queries. Note that since not all of the DOIs are indexd in WOS, we will retrieve fewer than 1000 results.

In [19]:
# We will specify the DOIs to query within a loop
base_parameters = '&'.join([
    'db=WOS',
    'limit=50',
    'page=1'
])

rows = []

# We will need to 20 requests to retrieve 1000 results
for i in range(20):
    # Extract DOIs in spans of 50
    query_dois = dois[i*50:(i+1)*50]
    query = f"q=(DO=({' OR '.join(query_dois)}))"

    url = f'https://api.clarivate.com/apis/wos-starter/v1/documents?{base_parameters}&{query}'
    doi_data = requests.get(url, headers=WOS_STARTER_HEADER).json()

    for hit in doi_data['hits']:
        doi = hit['identifiers']['doi']
        title = hit['title']
        journal = hit['source']['sourceTitle']
        year = hit['source']['publishYear']

        rows.append([doi, title, journal, year])
    sleep(1)

columns = ["DOI", "Title", "Journal", "Year"]
df = pd.DataFrame(rows, columns=columns)

df.head()

,DOI,Title,Journal,Year
0,10.1177/009385488000700401,CRIMINAL-JUSTICE AND BEHAVIOR - AN EDITORIAL R...,CRIMINAL JUSTICE AND BEHAVIOR,1980
1,10.2466/pr0.1982.51.2.663,THE SELF-EFFICACY SCALE - CONSTRUCTION AND VAL...,PSYCHOLOGICAL REPORTS,1982
2,10.1016/0022-1031(83)90023-9,PROTECTION MOTIVATION AND SELF-EFFICACY - A RE...,JOURNAL OF EXPERIMENTAL SOCIAL PSYCHOLOGY,1983
3,10.2307/1289308,STORYTELLING FOR OPPOSITIONISTS AND OTHERS - A...,MICHIGAN LAW REVIEW,1989
4,10.3102/00346543062002129,PROFESSIONAL GROWTH AMONG PRESERVICE AND BEGIN...,REVIEW OF EDUCATIONAL RESEARCH,1992


## 3. Short Record View with WOS Expanded

The Short Record view for the WOS Expanded API allows users to receive limited metadata without counting against an institution's usage quota for Full Record downloads. We recommend to first check whether the data you require is  More information can be found in the [WOS API release notes](https://clarivate.com/academia-government/release-notes/wos-apis/release-notes-may-23-2024/) and the [Swagger documentation](https://api.clarivate.com/swagger-ui/?url=https://developer.clarivate.com/apis/wos/swagger).

This example uses the `citing` endpoint of the WOS Expanded API to retrieve data for a given DOI.

The URL requires the following four parameters:

1. `databaseId` specifies what database to use. In this case, we are using the `WOS` (Web of Science) database.
2. `usrQuery` specifies the query we are using. The following parameters are used in this case:
    - `DO` specifies what DOI to search for.
3. `count` specifies the number of records to return.
4. `firstRecord` specifies the first record to return.

This example also includes the optional parameter `optionView`, which we set to `SR` to indicate that we would like to retrieve the Short Record view of this query. This parameter is set to `FR` by default, which returns the Full Record view.

In [20]:
doi = '10.1021/acsomega.8b01647'
endpoint = 'citing'

parameters = '&'.join([
    endpoint,
    'databaseId=WOS',
    f'usrQuery=(DO=({doi}))',
    'count=1',
    'firstRecord=1',
    'optionView=SR'
])

url = f'https://wos-api.clarivate.com/api/wos?{parameters}'
response = requests.get(url, headers=WOS_EXPANDED_HEADER)

# Status code 200 indicates success
response.status_code

200

In [21]:
sr_data = response.json()

# Extract data of the first article returned
sr_article_data = sr_data['Data']['Records']['records']['REC'][0]
pprint(sr_article_data, depth=2)

{'UID': 'WOS:000446186000103',
 'dynamic_data': {'citation_related': {...}, 'cluster_related': {...}},
 'links': {'citingArticles': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosexpandedua&SrcAuth=WosAPI&KeyUT=WOS:000446186000103&DestLinkType=CitingArticles&DestApp=WOS_CPL',
           'record': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosexpandedua&SrcAuth=WosAPI&KeyUT=WOS:000446186000103&DestLinkType=FullRecord&DestApp=WOS_CPL',
           'references': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosexpandedua&SrcAuth=WosAPI&KeyUT=WOS:000446186000103&DestLinkType=CitedReferences&DestApp=WOS',
           'related': 'https://www.webofscience.com/api/gateway?GWVersion=2&SrcApp=wosexpandedua&SrcAuth=WosAPI&KeyUT=WOS:000446186000103&DestLinkType=RelatedRecords&DestApp=WOS_CPL'},
 'r_id_disclaimer': 'ResearcherID data provided by Clarivate Analytics',
 'static_data': {'fullrecord_metadata': {...}, 'summary': {...}}}


As we can see in the output above, the results are primarily split between `static_data` and `dynamic_data`.

### Retrieving Static Metadata

The basic structure of the static metadata is given below:

In [22]:
pprint(sr_article_data['static_data'], depth=2)

{'fullrecord_metadata': {'normalized_doctypes': {...}, 'refs': {...}},
 'summary': {'doctypes': {...},
             'names': {...},
             'pub_info': {...},
             'titles': {...}}}


The `summary` contains basic data relating to the publication. We extract some of this data in the examples below.

In [23]:
# Print source and article titles
sr_article_data['static_data']['summary']['titles']['title']

[{'type': 'source', 'content': 'ACS OMEGA'},
 {'type': 'item',
  'content': 'How Precise Are Our Quantitative Structure-Activity Relationship Derived Predictions for New Query Chemicals?'}]

In [24]:
# Print authors
for name in sr_article_data['static_data']['summary']['names']['name']:
    print(name['role'], ':', name['display_name'])

author : Roy, Kunal
author : Ambure, Pravin
author : Kar, Supratik


In [25]:
# Print volume and page data
sr_article_data['static_data']['summary']['pub_info']

{'vol': 3, 'pubyear': 2018, 'pubtype': 'Journal', 'page': {'page_count': 15}}

### Retrieving Dynamic Data

The entirety of the `dynamic_data` of the Short Record is printed below. The `dynamic_data` will be more thoroughly explored in later examples using the Full Record view.

In [26]:
sr_article_data['dynamic_data']

{'citation_related': {'tc_list': {'tc_mod_date': '2025-02-05',
   'silo_tc': {'coll_id': 'WOS', 'local_count': 104}}},
 'cluster_related': {'identifiers': {'identifier': [{'type': 'issn',
     'value': '2470-1343'},
    {'type': 'doi', 'value': '10.1021/acsomega.8b01647'},
    {'type': 'pmid', 'value': 'MEDLINE:31459245'}]}}}

## 4. Full Record View with WOS Expanded

This example analyzes the Full Record view of the same article as the previous example. All parameters are the same as the previous example except for `optionView`, which is set to `FR` (the default value).

In [27]:
doi = '10.1021/acsomega.8b01647'
endpoint = 'citing'

parameters = '&'.join([
    endpoint,
    'databaseId=WOS',
    f'usrQuery=(DO=({doi}))',
    'count=1',
    'firstRecord=1',
    'optionView=FR'
])

url = f'https://wos-api.clarivate.com/api/wos?{parameters}'
response = requests.get(url, headers=WOS_EXPANDED_HEADER)

# Status code 200 indicates success
response.status_code

200

In [28]:
fr_data = response.json()

# Extract data of the first article returned
fr_article_data = fr_data['Data']['Records']['records']['REC'][0]
pprint(fr_article_data, depth=2)

{'UID': 'WOS:000446186000103',
 'dates': {'date_created': '2018-10-16T08:20:52.724758',
           'date_loaded': '2018-09-01T23:59:59.00000',
           'date_modified': '2020-09-22T11:27:33.607987'},
 'dynamic_data': {'citation_related': {...},
                  'cluster_related': {...},
                  'wos_usage': {...}},
 'r_id_disclaimer': 'ResearcherID data provided by Clarivate Analytics',
 'static_data': {'contributors': {...},
                 'fullrecord_metadata': {...},
                 'item': {...},
                 'summary': {...}}}


### Retrieving Static Metadata

The `static_data` contains the following information:

In [29]:
pprint(fr_article_data['static_data'], depth=2)

{'contributors': {'contributor': [...], 'count': 4},
 'fullrecord_metadata': {'abstracts': {...},
                         'addresses': {...},
                         'category_info': {...},
                         'fund_ack': {...},
                         'languages': {...},
                         'normalized_doctypes': {...},
                         'normalized_languages': {...},
                         'refs': {...},
                         'reprint_addresses': {...}},
 'item': {'bib_id': '3 (9): 11392-11406 SEP 2018',
          'bib_pagecount': {...},
          'coll_id': 'WOS',
          'ids': {...},
          'keywords_plus': {...},
          'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
          'xsi:type': 'itemType_wos'},
 'summary': {'EWUID': {...},
             'doctypes': {...},
             'names': {...},
             'pub_info': {...},
             'publishers': {...},
             'titles': {...}}}


Now, we can extract some information out of the static data.

In [30]:
# Extract contributors' names and ORCID ID (if available)
for contributor in fr_article_data['static_data']['contributors']['contributor']:
    if 'orcid_id' in contributor['name']:
        print(f"{contributor['name']['display_name']}: {contributor['name']['orcid_id']}")
    else:
        print(f"{contributor['name']['display_name']}")

AMBURE, PRAVIN
Roy, Kunal
Roy, Kunal: 0000-0003-4486-8074
Kar, Supratik: 0000-0002-9411-2091


In [31]:
# Print the funding acknowledgement data
fr_article_data['static_data']['fullrecord_metadata']['fund_ack']['grants']

{'count': 1,
 'grant': {'grant_agency_names': [{'pref': 'Y',
    'content': 'University Grants Commission, India'},
   {'pref': 'N', 'content': 'UGC, New Delhi'}],
  'grant_agency': 'UGC, New Delhi'}}

In [32]:
# Print publisher data
fr_article_data['static_data']['summary']['publishers']['publisher']

{'names': {'count': 1,
  'name': {'seq_no': 1,
   'role': 'publisher',
   'full_name': 'AMER CHEMICAL SOC',
   'unified_name': 'Amer Chemical Soc',
   'addr_no': 1,
   'display_name': 'AMER CHEMICAL SOC'}},
 'address_spec': {'city': 'WASHINGTON',
  'addr_no': 1,
  'full_address': '1155 16TH ST, NW, WASHINGTON, DC 20036 USA'}}

### Retrieving Dynamic Metadata

The `dynamic_data` contains the following information:

In [33]:
pprint(fr_article_data['dynamic_data'], depth=2)

{'citation_related': {'SDG': {...},
                      'citation_topics': {...},
                      'tc_list': {...},
                      'tc_list_cc': {...}},
 'cluster_related': {'identifiers': {...}},
 'wos_usage': {'alltime': 14, 'last180days': 1}}


In [34]:
# Print subjects of related citations
fr_article_data['dynamic_data']['citation_related']['citation_topics']['subj-group']['subject']

[{'content-type': 'macro', 'content-id': '2', 'content': 'Chemistry'},
 {'content-type': 'meso',
  'content-id': '2.123',
  'content': 'Protein Structure, Folding & Modelling'},
 {'content-type': 'micro', 'content-id': '2.123.778', 'content': 'QSAR'}]

In [35]:
# Print identifiers
fr_article_data['dynamic_data']['cluster_related']['identifiers']['identifier']

[{'type': 'issn', 'value': '2470-1343'},
 {'type': 'doi', 'value': '10.1021/acsomega.8b01647'},
 {'type': 'pmid', 'value': 'MEDLINE:31459245'}]

In [36]:
for db in fr_article_data['dynamic_data']['citation_related']['tc_list']['silo_tc']:
    if db['coll_id'] == 'WOS':
        print(f"There are {db['local_count']} citations in the WOS database.")

There are 104 citations in the WOS database.


## 5. Retrieving References from an Article (WOS Expanded)

In this example, we use the `references` endpoint to retrieve the articles that a given article cites.

In [37]:
# Use the WOS ID of the article from the previous example
wos_id = fr_article_data["UID"]

parameters = '&'.join([
    'databaseId=WOS',
    f'uniqueId={wos_id}',
    'count=100',
    'firstRecord=1'
])

url = f"https://wos-api.clarivate.com/api/wos/references?{parameters}"
ref_data = requests.get(url, headers=WOS_EXPANDED_HEADER).json()

print(f'{ref_data["QueryResult"]["RecordsFound"]} records found')

43 records found


In [38]:
ref_data['Data'][0]

{'UID': 'WOS:000425492800015',
 'CitedAuthor': 'Gajewicz, A.',
 'TimesCited': '56',
 'Year': '2018',
 'Page': '408',
 'Volume': '5',
 'CitedWork': 'ENVIRONMENTAL SCIENCE-NANO',
 'CitedTitle': "How to judge whether QSAR/read-across predictions can be trusted: a novel approach for establishing a model's applicability domain",
 'DOI': '10.1039/c7en00774d'}

In [39]:
# Print data for the first 3 results
categories = ["CitedAuthor", "CitedTitle", "CitedWork", "Year", "Volume", "Page", "DOI"]
for article in ref_data["Data"][:3]:
    line = ''
    for category in categories:
        # Append data to string if it exists, otherwise append nothing
        line += article.get(category, '') + ', '
    # Remove extra ', ' from end
    line = line[:-2]
    print(line)

Gajewicz, A., How to judge whether QSAR/read-across predictions can be trusted: a novel approach for establishing a model's applicability domain, ENVIRONMENTAL SCIENCE-NANO, 2018, 5, 408, 10.1039/c7en00774d
Liu, Ruifeng, General Approach to Estimate Error Bars for Quantitative Structure-Activity Relationship Predictions of Molecular Activity, JOURNAL OF CHEMICAL INFORMATION AND MODELING, 2018, 58, 1561, 10.1021/acs.jcim.8b00114
Roy, Kunal, Is it possible to improve the quality of predictions from an "intelligent" use of multiple QSAR/QSPR/QSTR models?, JOURNAL OF CHEMOMETRICS, 2018, 32, , 10.1002/cem.2992


## 6. Retrieving Citations from an Article (WOS Expanded)

In this example, we'll use the `citing` endpoint to retrieve the articles that cite a given article.

In [40]:
endpoint = 'citing'
parameters = '&'.join([
    'databaseId=WOS',
    f'uniqueId={wos_id}',
    'optionView=FR',     # Retrieve the Full Record (goes against Full Record request limit)
    'count=100',
    'firstRecord=1'
])

url = f"https://wos-api.clarivate.com/api/wos/{endpoint}?{parameters}"
fr_citation_data = requests.get(url, headers=WOS_EXPANDED_HEADER).json()

# Display results of query
print(f'{fr_citation_data["QueryResult"]["RecordsFound"]} records found')

101 records found


In [41]:
pprint(fr_citation_data['Data']['Records']['records']['REC'][0], depth=2)

{'UID': 'WOS:001403397400001',
 'dates': {'date_created': '2025-01-29T13:55:07.179355',
           'date_loaded': '2025-01-29T23:59:59.00000',
           'date_modified': '2025-01-29T13:55:07.179355'},
 'dynamic_data': {'citation_related': {...},
                  'cluster_related': {...},
                  'wos_usage': {...}},
 'r_id_disclaimer': 'ResearcherID data provided by Clarivate Analytics',
 'static_data': {'contributors': {...},
                 'fullrecord_metadata': {...},
                 'item': {...},
                 'summary': {...}}}


In [42]:
pprint(fr_citation_data['Data']['Records']['records']['REC'][0]['static_data'], depth=2)

{'contributors': {'contributor': {...}, 'count': 1},
 'fullrecord_metadata': {'abstracts': {...},
                         'addresses': {...},
                         'category_info': {...},
                         'fund_ack': {...},
                         'keywords': {...},
                         'languages': {...},
                         'normalized_doctypes': {...},
                         'normalized_languages': {...},
                         'refs': {...},
                         'reprint_addresses': {...}},
 'item': {'bib_id': '279: - FEB 2025',
          'bib_pagecount': {...},
          'coll_id': 'WOS',
          'ids': {...},
          'keywords_plus': {...},
          'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
          'xsi:type': 'itemType_wos'},
 'summary': {'EWUID': {...},
             'doctypes': {...},
             'names': {...},
             'pub_info': {...},
             'publishers': {...},
             'titles': {...}}}


In [43]:
# Print data for the first 4 citations
for citation in fr_citation_data["Data"]["Records"]["records"]["REC"][:3]:
    first_author = title = source = year = volume = page = ""

    # Grab the name of the first author listed
    first_author = citation["static_data"]["summary"]["names"]["name"][0]["full_name"]
    
    # Grab source and article titles
    for metadatum in citation["static_data"]["summary"]["titles"]["title"]:
        if metadatum["type"] == "item":
            title = metadatum["content"]
        elif metadatum["type"] == "source":
            source = metadatum["content"]
    
    # Grab year and volume
    year = citation["static_data"]["summary"]["pub_info"]["pubyear"]
    volume = citation["static_data"]["summary"]["pub_info"].get("vol", "")  # Leave empty if no Volume given
    page = citation["static_data"]["summary"]["pub_info"]["page"].get("begin", "")  # Leave empty if no Page given

    print(f"{first_author}, {title}, {source}, {year}, {volume}, {page}")

Pore, Souvik, Intelligent consensus-based predictions of early life stage toxicity in fish tested in compliance with OECD Test Guideline 210, AQUATIC TOXICOLOGY, 2025, 279, 
El Yaqoubi, Mohamed, Pyridopyrazine derivatives as highly selective histamine H4 receptor antagonist for the treatment of atopic dermatitis: QSAR modeling and molecular docking studies, AIMS ALLERGY AND IMMUNOLOGY, 2024, 8, 303
Chen, Shuo, High-throughput prediction of oral acute toxicity in<i> Rat</i> and<i> Mouse</i> of over 100,000 polychlorinated persistent organic pollutants (PC-POPs) by interpretable data fusion-driven machine learning global models, JOURNAL OF HAZARDOUS MATERIALS, 2024, 480, 
